In [ ]:
pip install textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.9 MB/s eta 0:00:00


In [ ]:
import nltk
import warnings
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, RNN
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.models import Model
from nltk.tokenize import word_tokenize
warnings.filterwarnings("ignore")
from textblob import TextBlob
from nltk.util import ngrams
import matplotlib.cm as cm
import matplotlib as mpl
from tqdm import tqdm
import pandas as pd
import numpy as np
import zipfile
import random
import pickle
import re

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tokenizer_enc = pickle.load(open(r"tokenizer_encoder.pkl", "rb"))
tokenizer_dec = pickle.load(open(r"tokenizer_decoder.pkl", "rb"))

train_enc_inp = pickle.load(open(r"train_encoder_input.pkl", "rb"))
val_enc_inp = pickle.load(open(r"validation_encoder_input.pkl", "rb"))
test_enc_inp = pickle.load(open(r"test_encoder_input.pkl", "rb"))

train_dec_inp = pickle.load(open(r"train_decoder_input.pkl", "rb"))
val_dec_inp = pickle.load(open(r"validation_decoder_input.pkl", "rb"))
test_dec_inp = pickle.load(open(r"test_decoder_input.pkl", "rb"))

train_dec_out = pickle.load(open(r"train_decoder_output.pkl", "rb"))
val_dec_out = pickle.load(open(r"validation_decoder_output.pkl", "rb"))
test_dec_out = pickle.load(open(r"test_decoder_output.pkl", "rb"))

In [ ]:
class Encoder(tf.keras.Model):


    def __init__(self, vocab_size, output_dim, enc_units, input_length):
        super().__init__()
        self.vocab_size = vocab_size
        self.output_dim = output_dim
        self.input_length = input_length
        self.enc_units = enc_units
        self.enc_output = 0
        self.enc_state_h = 0
        self.enc_state_c = 0

        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.output_dim,
                                   input_length=self.input_length, mask_zero=True, name="embedding_layer_encoder")

        self.enc = LSTM(self.enc_units, return_state=True, return_sequences=True, name="Encoder_LSTM")

    def call(self,input_sequence, states):
        input_embedding = self.embedding(input_sequence)
        self.enc_output, self.enc_state_h, self.enc_state_c = self.enc(input_embedding, initial_state=states)
        return self.enc_output, self.enc_state_h, self.enc_state_c

    def initialize_states(self, batch_size):
        ini_hidden_state = tf.zeros(shape=(batch_size, self.enc_units))
        ini_cell_state = tf.zeros(shape=(batch_size, self.enc_units))
        return ini_hidden_state, ini_cell_state

In [ ]:
class Decoder(tf.keras.Model):


    def __init__(self, vocab_size, output_dim, dec_units, input_length):
        super().__init__()
        self.vocab_size = vocab_size
        self.output_dim = output_dim
        self.dec_units = dec_units
        self.input_length = input_length

        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.output_dim,
                                   input_length=self.input_length, mask_zero=True, name="embedding_layer_decoder")

        self.dec = LSTM(self.dec_units, return_sequences=True, return_state=True, name="Decoder_LSTM")

    def call(self,input_sequence, initial_states):
        target_embedding = self.embedding(input_sequence)
        dec_output, dec_state_h, dec_state_c = self.dec(target_embedding, initial_state=initial_states)
        return dec_output, dec_state_h, dec_state_c

In [ ]:
class Encoder_decoder(tf.keras.Model):


    def __init__(self, enc_vocab_size, enc_output_dim, enc_inp_length, enc_units,
                 dec_vocab_size, dec_output_dim, dec_inp_length, dec_units, batch_size):

        super().__init__()
        self.encoder = Encoder(vocab_size=enc_vocab_size, output_dim=enc_output_dim,
                               input_length=enc_inp_length, enc_units=enc_units)
        self.decoder = Decoder(vocab_size=dec_vocab_size, output_dim=dec_output_dim,
                               input_length=dec_inp_length, dec_units=dec_units)
        self.dense   = Dense(dec_vocab_size, activation='softmax')
        self.ini_states = self.encoder.initialize_states(batch_size=batch_size)

    def call(self,data):
        input,output = data[0], data[1]
        enc_output, enc_h,enc_c = self.encoder(input, self.ini_states)
        dec_output, dec_h,dec_c = self.decoder(output, [enc_h, enc_c])
        output = self.dense(dec_output)
        return output

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [ ]:
enc_voc_size = len(tokenizer_enc.word_index) + 1
dec_voc_size = len(tokenizer_dec.word_index) + 1
embedd_dim = 100
enc_inp_len = 12
dec_inp_len = 13
lstm_size=64
batch_size=1024

In [ ]:
train_trunc_idx = (train_enc_inp.shape[0]//batch_size)*batch_size
val_trunc_idx = (val_enc_inp.shape[0]//batch_size)*batch_size

train_enc_inp_truncated = train_enc_inp[:train_trunc_idx]
train_dec_inp_truncated = train_dec_inp[:train_trunc_idx]
train_dec_out_truncated = train_dec_out[:train_trunc_idx]

val_enc_inp_truncated = val_enc_inp[:val_trunc_idx]
val_dec_inp_truncated = val_dec_inp[:val_trunc_idx]
val_dec_out_truncated = val_dec_out[:val_trunc_idx]

In [ ]:
model  = Encoder_decoder(enc_vocab_size=enc_voc_size,enc_output_dim=embedd_dim,
                         enc_inp_length=enc_inp_len,enc_units=lstm_size,
                         dec_vocab_size=dec_voc_size,dec_output_dim=embedd_dim,
                         dec_inp_length=dec_inp_len,dec_units=lstm_size,
                         batch_size=batch_size)

log_dir=f'/lstm/Logs/'
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, verbose=1, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_delta=0.001, patience=5, verbose=1, factor=0.9)
check_point = ModelCheckpoint('/lstm/model/', monitor='val_loss',
                              save_best_only=True, save_weights_only=True, mode='min', verbose=0)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss=loss_function)

In [ ]:
model.fit(x=[train_enc_inp_truncated, train_dec_inp_truncated], y=train_dec_out_truncated,
          validation_data=([val_enc_inp_truncated, val_dec_inp_truncated], val_dec_out_truncated),
          epochs=30, batch_size=batch_size, callbacks=[early_stop, reduce_lr, check_point])

Epoch 1/30
392/392 [==============================] - 285s 695ms/step - loss: 5.7842 - val_loss: 5.0917 - lr: 0.0010
Epoch 2/30
392/392 [==============================] - 235s 600ms/step - loss: 4.9633 - val_loss: 4.7120 - lr: 0.0010
Epoch 3/30
392/392 [==============================] - 226s 577ms/step - loss: 4.5148 - val_loss: 4.2046 - lr: 0.0010
Epoch 4/30
392/392 [==============================] - 224s 571ms/step - loss: 4.0425 - val_loss: 3.8122 - lr: 0.0010
Epoch 5/30
392/392 [==============================] - 227s 579ms/step - loss: 3.7223 - val_loss: 3.5691 - lr: 0.0010
Epoch 6/30
392/392 [==============================] - 222s 567ms/step - loss: 3.4821 - val_loss: 3.3647 - lr: 0.0010
Epoch 7/30
392/392 [==============================] - 223s 568ms/step - loss: 3.2672 - val_loss: 3.1904 - lr: 0.0010
Epoch 8/30
392/392 [==============================] - 221s 565ms/step - loss: 3.0812 - val_loss: 3.0460 - lr: 0.0010
Epoch 9/30
392/392 [==============================] - 226s 576ms

In [ ]:
model.summary()

Model: "encoder_decoder_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_1 (Encoder)         multiple                  9240440   
                                                                 
 decoder_1 (Decoder)         multiple                  7826340   
                                                                 
 dense_1 (Dense)             multiple                  5059665   
                                                                 
Total params: 22126445 (84.41 MB)
Trainable params: 22126445 (84.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
correct_word_idx_dictionary = tokenizer_dec.word_index
correct_idx_word_dictionary = {v: k for k, v in correct_word_idx_dictionary.items()}

def predict(input_sentence):
    input_sentence = tokenizer_enc.texts_to_sequences([input_sentence])
    initialize_hidden_state = tf.zeros([1, 64])
    initialize_cell_state = tf.zeros([1, 64])
    initialize_states = [initialize_hidden_state, initialize_cell_state]
    pred_total = []
    encoder_output, encoder_state_h, encoder_state_c = model.layers[0](np.expand_dims(input_sentence[0], 0), ini_states)
    states_values = [encoder_state_h, encoder_state_c]
    pred = []
    sentence = []
    cur_vector = np.array([[1]])
    for i in range(12):
        decoder_output, decoder_state_h, decoder_state_c = model.layers[1](cur_vec, states_values)
        output = model.layers[2](decoder_output)
        states_values = [decoder_state_h, decoder_state_c]
        cur_vector = np.reshape(np.argmax(output), (1, 1))
        if correct_idx_word_dict[cur_vec[0][0]] == '<end>':
            break
        pred.append(cur_vector[0][0])
    for i in pred:
        sentence.append(correct_idx_word_dictionary[i])
    return " ".join(sentence)

In [ ]:
model.save('/lstm/model/')